In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np 
import faiss


h:\RUTUBE_HACK\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
model_path = "./winstrike_rutube_tags_model" # путь до папки с моделью SentenceTransformer
model = SentenceTransformer(model_path, )
dim = 768 # размер вектора эмбеддинга

СОХРАНЕНИЕ EMBENDDING'ОВ ТЕГОВ(Запускать если отсутствует файл tags_embeddings.npy)

In [3]:
# обозначение тайтлов тегов
lvl_1_label = "Уровень 1 (iab)"
lvl_2_label = "Уровень 2 (iab)"
lvl_3_label = "Уровень 3 (iab)"

In [4]:
# Читаем csv-файл с тегами
tags_df = pd.read_csv("./IAB_tags.csv").fillna("")
# Фильтруем полностью пустые строки
tags_df = tags_df.loc[(tags_df[lvl_1_label] != "") | (tags_df[lvl_2_label] != "") | (tags_df[lvl_3_label] != "")]

tags_list = []

with open("./organized_tags.txt", mode="w", encoding='utf-8') as file:
    for row in tags_df.iterrows():
        tag = row[1][lvl_1_label]
        if row[1][lvl_2_label] != "":
            tag = tag + ": " + row[1][lvl_2_label]
        else:
            tags_list.append(tag)
            continue    
        if row[1][lvl_3_label] != "":
            tag = tag + ": " + row[1][lvl_3_label]
        else:
            tags_list.append(tag)
            continue
        tags_list.append(tag)

    file.write("\n".join(tags_list))

In [ ]:
# преобразование строкового представления тега в вектор чисел (embedding)
embedding_tags_list = []

for str_tag in tags_list:
    embedding_tags_list.append(model.encode(str_tag, convert_to_tensor=True).cpu().numpy())

embedding_tags_list

In [17]:
# сохранение векторов тегов в .npy файл для последующего использования
np_embedding_tags_list = np.array(embedding_tags_list, dtype=np.float64)

with open("./tags_embeddings.npy", "wb") as file:
    np.save(file, np_embedding_tags_list)

ФУНКЦИЯ ПОЛУЧЕНИЯ ТЕГОВ ПО НАЗВАНИЮ И ОПИСАНИЮ ВИДЕО

In [3]:
# Чтение embedding тегов
with open('./tags_embeddings.npy', "rb") as f:
    np_tags_arr = np.load(f)

In [4]:
# Чтение всех тегов
with open('./organized_tags.txt', "r", encoding="utf-8") as f:
    all_tags_list = f.read().split('\n')

In [5]:
def get_predict_from_title_and_description(title : str, description : str) -> list:
    """ Получение тегов к видео по наименованию и описанию """
    global model, dim, np_tags_arr
    
    embedding = model.encode(title + " " + description, convert_to_tensor=True).cpu().numpy()
    index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
    index.add(np_tags_arr)


    topn = 3
    scores, predictions = index.search(np.array([list(embedding)], dtype=np.float64), topn)


    return scores[0], [all_tags_list[idx] for idx in predictions[0]]


In [6]:
get_predict_from_title_and_description("МАКСИМ НАРОДНЫЙ", "Выпуск №43 ГОТОВИМ СЭНДВИЧ С КИРИЛЛОМ ФОКИНЫМ-САЗОНОВЫМ,Ведущий канала Максим и приглашенный гость популярный блогер Кирилл Фокин-Сазонов предлагают своим подписчикам приготовить сэндвич. Главный герой делает акцент на том, что это простое блюдо можно приготовить быстро и бюджетно.")

(array([346.06934, 314.75943, 313.5791 ], dtype=float32),
 ['Еда и напитки: Кулинария',
  'Массовая культура',
  'Массовая культура: Юмор и сатира'])

In [9]:
def get_tags(title : str, description : str, video_path : str):
    """ Получение тегов к видео """
    # Получение тегов всех уровней по описанию
    scores, tags_from_text = get_predict_from_title_and_description(title, description)
    # Получение тегов первого уровня по видеоряду
    tags_from_video = []

    result_tags = []

    # отфильтровываем теги, на которых модель сомневается
    tags_from_text = [tag for (idx, tag) in enumerate(tags_from_text) if int(scores[idx]) > 250]

    if not len(tags_from_text):
        return tags_from_video

    # сопоставление полученных тегов, поиск пересечений
    for tag_from_text in tags_from_text:
        for tag_from_video in tags_from_video:
            if (tag_from_video in tag_from_text):
                result_tags.append(tag_from_text)
                break
    
    if not len(result_tags):
        return tags_from_text
    
    return result_tags

ВАРИАНТ ДЛЯ BACKEND'А

In [7]:
# def get_tags(title : str, description : str, lvl_1_tags : list[str]) -> list[str]:
#     scores, tags_from_text = get_predict_from_title_and_description(title, description)

#     result_tags = []

#     # отфильтрорываем теги, на которых модель сомневается
#     tags_from_text = [tag for (idx, tag) in enumerate(tags_from_text) if int(scores[idx]) > 250]

#     if not len(tags_from_text):
#             return lvl_1_tags

#     for tag_from_text in tags_from_text:
#         for tag_from_video in lvl_1_tags:
#             if (tag_from_video in tag_from_text):
#                 result_tags.append(tag_from_text)
#                 break
    
#     if not len(result_tags):
#         return tags_from_text
    
#     return result_tags

SyntaxError: incomplete input (2597908440.py, line 1)

In [12]:
#Пример если модель не уверна
get_tags("asdasdasd.  dasdsad", "asdsadasdad", "")

[]

In [11]:
get_tags("МАКСИМ НАРОДНЫЙ", "Выпуск №43 ГОТОВИМ СЭНДВИЧ С КИРИЛЛОМ ФОКИНЫМ-САЗОНОВЫМ,Ведущий канала Максим и приглашенный гость популярный блогер Кирилл Фокин-Сазонов предлагают своим подписчикам приготовить сэндвич. Главный герой делает акцент на том, что это простое блюдо можно приготовить быстро и бюджетно.", "")

['Еда и напитки: Кулинария',
 'Массовая культура',
 'Массовая культура: Юмор и сатира']